<a href="https://colab.research.google.com/github/anmolg0/ajackus_assignment/blob/main/Source_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sqlite3
import spacy
import re

# Load the spaCy model for natural language processing
nlp = spacy.load("en_core_web_sm")

# Connect to the SQLite database
conn = sqlite3.connect('company.db')
cursor = conn.cursor()

def fetch_data(query):
    patterns = [
        (r"show me all employees in the (\w+) department", "SELECT Name FROM Employees WHERE Department = ?"),
        (r"who is the manager of the (\w+) department", "SELECT Manager FROM Departments WHERE Name = ?"),
        (r"list all employees hired after (\d{4}-\d{2}-\d{2})", "SELECT Name FROM Employees WHERE Hire_Date > ?"),
        (r"what is the total salary expense for the (\w+) department", "SELECT SUM(Salary) FROM Employees WHERE Department = ?")
    ]

    for pattern, sql_query in patterns:
        match = re.search(pattern, query, re.IGNORECASE)
        if match:
            params = match.groups()
            try:
                cursor.execute(sql_query, params)
                results = cursor.fetchall()
                if not results:
                    return "No results found."
                return results
            except sqlite3.Error as e:
                return f"Database error: {e}"

    return "I'm sorry, I didn't understand the query. Please try again."

def chat_assistant():
    print("Chat Assistant: How can I help you?")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Chat Assistant: Goodbye!")
            break
        data = fetch_data(user_input)
        if isinstance(data, str):  # If the response is an error or message
            print(f"Chat Assistant: {data}")
        else:  # If the response is actual data
            for row in data:
                print(f"Chat Assistant: {row[0]}")

if __name__ == "__main__":
    chat_assistant()

# Close the connection when done
conn.close()


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Chat Assistant: How can I help you?
You: Show me all employees in the Sales department.
Chat Assistant: Alice
You: Who is the manager of the Engineering department?
Chat Assistant: Bob
You: exit
Chat Assistant: Goodbye!
